<a href="https://colab.research.google.com/github/chunhaoc1/Automate-Research-paper/blob/main/autoreader_c3d_01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Introduction**

This program aim to automate the process of searching and categories article or paper on "CAREN" AND "Motek Medical" field with the key question how has the Motek Medical CAREN system been used in research and clinical applications over the last five years.

We are necessary to learn:
How to fetch the particular data on google schalor.
Categories those information into table format.
How to request chatpgt to assist us finalize the research paper table.


Belows' steps depict the

1.   install scholarly and openAI
2.   Python atomatic reach paper reader
3.   Categories those data into table
4.   Analyzing table with Chatgpt





# **Authors:**

Dr. Hossein Mokhtarzadeh


Xin Shen - Master of Information Technology, University of Melbourne

xsshen2@student.unimelb.edu.au

Chun-Hao Chen - Master of Information Systems, University of Melbourne

chunhaoc1@student.unimelb.edu.au

Songxiang Tang - Master of Information Systems, University of Melbourne

songxiangt@student.unimelb.edu.au





 **Step1**: Install scholarly to capture data from Google Scholar

In [ ]:
pip install scholarly

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.1/55.1 kB 896.4 kB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.4/66.4 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.4/9.4 MB 29.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 50.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.1/121.1 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 476.0/476.0 kB 24.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 1.4 MB/s eta 0:00:00
  Created wheel for bibtexparser: filename=bibtexparser-1.4.1-py3-none-any.whl size=43253 sha256=869c25831c7d2c9a04459eec1c148ab6d1f956072469759af5ff82a64e912085
  Stored in directory: /root/.cache/pip/wheels/

**Step1: Install openAI**

In [ ]:
pip install openai==0.28

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 1.3 MB/s eta 0:00:00


**Step2 : Python atomatic reach paper reader**

**Step3 : Categories those data into table**

In [ ]:
from ctypes import sizeof
from scholarly import scholarly
import openai
import csv

openai.api_key='Upload your own API key'

def get_filtered_articles(topic, start_year, end_year,t):
    search_query = scholarly.search_pubs(topic)
    articles = []

    for article in search_query:
        # Check the publish year for the artical
        pub_year_str = article['bib'].get('pub_year', '0')
        pub_year = int(pub_year_str) if pub_year_str.isdigit() else 0

        # Show progress
        num=len(articles);
        print(f"Progress: {num}/{t}")
        print(f"Found article: {article['bib']['title']} ({pub_year})")
        if len(articles)>=t:
          break;

        # filter targeted article
        if start_year <= pub_year <= end_year:
            abstract = article['bib'].get('abstract', 'No abstract available')
            articles.append({
                'title': article['bib']['title'],
                'abstract': abstract,
                'year': pub_year,
                'url': article.get('pub_url', 'No URL available')
            })

    return articles


def analyze_with_chatgpt(abstract):

    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "user", "content": f"Please review the following research paper about CAREN and summarize it in the table form with alttribute [Year, research field,Title, Clinical application, Methodology, outcomes]. (Note: please keep the answer brief:):\n{abstract}"}
          ]
    )
    return response.choices[0].message['content']



def save2Csv(table_data_list, csv_file):
    """
    Save a list of table strings to a CSV file.

    :param table_data_list: List of table strings formatted like Markdown tables.
    :param csv_file: Name of the output CSV file (default is "research_articles.csv").
    """
    with open(csv_file, mode='w', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)

        # Loop through each table in the list
        for table_data in table_data_list:
            # Parse the table into rows of data
            lines = table_data.strip().split("\n")

            # Extract headers and data
            headers = [header.strip() for header in lines[0].strip("|").split("|")]
            rows = []
            for line in lines[2:]:  # Skip the header and separator lines
                row = [cell.strip() for cell in line.strip("|").split("|")]
                rows.append(row)

            # Write headers only for the first table
            if file.tell() == 0:
                writer.writerow(headers)

            # Write rows
            writer.writerows(rows)

    print(f"Data successfully saved to {csv_file}")

def save2Text(table_data_list, text_file="research_articles.txt"):
    """
    Save a list of table strings to a text file.

    :param table_data_list: List of table strings formatted like Markdown tables.
    :param text_file: Name of the output text file (default is "research_articles.txt").
    """
    with open(text_file, mode='w', encoding='utf-8') as file:
        # Loop through each table in the list
        for table_data in table_data_list:
            # Write the table data to the text file
            file.write(table_data.strip() + "\n\n")

    print(f"Data successfully saved to {text_file}")

def save2Textt(table_data_list, text_file="research_articles.txt"):
    """
    Save a list of dictionaries representing table rows to a text file.

    :param table_data_list: List of dictionaries where each dictionary represents a row in the table.
    :param text_file: Name of the output text file (default is "research_articles.txt").
    """
    with open(text_file, mode='w', encoding='utf-8') as file:
        # Write headers based on the first dictionary's keys
        headers = list(table_data_list[0].keys())
        header_line = "| " + " | ".join(headers) + " |"
        separator_line = "| " + " | ".join(["-" * len(header) for header in headers]) + " |"

        # Write headers and separator to the text file
        file.write(header_line + "\n")
        file.write(separator_line + "\n")

        # Loop through each dictionary in the list and write its values as a table row
        for row_dict in table_data_list:
            row_line = "| " + " | ".join(str(row_dict[key]) for key in headers) + " |"
            file.write(row_line + "\n")

        file.write("\n")

    print(f"Data successfully saved to {text_file}")

def main():
    topic = "computer assisted rehabilitation environment"
    start_year = 2021
    end_year = 2024
    rslt=[]

    articles = get_filtered_articles(topic, start_year, end_year ,15)
    save2Textt(articles, "./articles.txt")

    if not articles:
        print("No articles found in the specified date range.")
    else:
        for article in articles:
            print(f"Title: {article['title']}")
            print(f"Year: {article['year']}")
            print("Abstract:", article['abstract'])
            parsed=analyze_with_chatgpt(article['abstract'])
            print(parsed)
            rslt.append(parsed)
            print("URL:", article['url'])
            print("\n" + "="*80 + "\n")
    save2Csv(rslt, "./research_articles.csv")
    save2Text(rslt, "./research_articles.txt")

if __name__ == "__main__":
    main()

Progress: 0/15
Found article: A systematic literature review of the use and effectiveness of the Computer Assisted Rehabilitation Environment for research and rehabilitation as it relates to the … (2015)
Progress: 0/15
Found article: CAREN-computer assisted rehabilitation environment (1999)
Progress: 0/15
Found article: Recent developments in computer assisted rehabilitation environments (2014)
Progress: 0/15
Found article: Comparison of walking overground and in a Computer Assisted Rehabilitation Environment (CAREN) in individuals with and without transtibial amputation (2012)
Progress: 0/15
Found article: The use of a computer-assisted rehabilitation environment (CAREN) for enhancing wounded warrior rehabilitation regimens (2013)
Progress: 0/15
Found article: … with neurological disorders benefit from immersive virtual reality? A scoping review on the emerging use of the computer-assisted rehabilitation environment (2024)
Progress: 1/15
Found article: The effect of Computer Assisted 

**Step4: Analyzing table with Chatgpt**

In [ ]:
# TEST the GPT api
import openai
import csv
openai.api_key='sk-C4WOxj1Xf49lMAFTJzQvk5saB-8uh-a4-R2CPrFdDOT3BlbkFJ7V3yb_qR2_5PS5N66W1YcbYRpOJegt1VyVDMNwlwkA'

def analyze_with_chatgpt(test):
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",  # Use GPT-4 or another model version
        messages=[
            {"role": "user", "content": f"hi i'm testing my code, please say something: {test}"}
        ]
    )

    return response.choices[0].message['content']

def save2Csv(table_data, csv_file):

  # Parse the table into rows of data
  lines = table_data.strip().split("\n")

  # Extract headers and data
  headers = [header.strip() for header in lines[0].strip("|").split("|")]
  rows = []
  for line in lines[2:]:  # Skip the header and separator lines
      row = [cell.strip() for cell in line.strip("|").split("|")]
      rows.append(row)

  # Define CSV file name
  csv_file = "research_articles.csv"

  # Write the parsed data to a CSV file
  with open(csv_file, mode='w', newline='', encoding='utf-8') as file:
      writer = csv.writer(file)

      # Write headers
      writer.writerow(headers)

      # Write rows
      writer.writerows(rows)

  print(f"Data successfully saved to {csv_file}")


test="Please give me a random table contain alttribute year, research field,Title, Clinical application, Methodology,outcomes,2 rows, please be concise"
content=analyze_with_chatgpt(test)
print(content)
save2Csv(content,"./test.csv")

Here is a random table:

| Year | Research Field | Title  | Clinical Application | Methodology | Outcomes |
|------|----------------|--------|----------------------|-------------|----------|
| 2020 | Neuroscience    | Study A | Alzheimer's disease  | MRI imaging | Improved cognitive function |
| 2019 | Oncology       | Study B | Breast cancer        | Drug therapy| Increased survival rates |
Data successfully saved to research_articles.csv
